In [1]:
import pandas as pd
from selenium import webdriver

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

import time
import os

from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

import csv
import pandas as pd

In [2]:
navigator = webdriver.Chrome(ChromeDriverManager().install())
navigator.get(os.getenv('URL'))

/tmp/ipykernel_29680/1960473131.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  navigator = webdriver.Chrome(ChromeDriverManager().install())


In [3]:
def click(path, type='xpath'):
    if type == 'xpath':
        navigator.find_element(By.XPATH, path).click()
    if type == 'name':
        navigator.find_element(By.NAME, path).click()

In [4]:
def translate_title(title):
    if title == 'Ano':
        return 'YEAR'
    if title == 'Km':
        return 'KM'
    if title == 'Combustível':
        return 'FUEL'  
    if title == 'Cor':
        return 'COLOR'
    if title == 'Estimativa de mercado':
        return 'FIPE'
    if title == 'Pátio':
        return 'YARD'
    if title == 'Retomada':
        return 'RESUME-TYPE'
    if title == 'Tipo':
        return 'TYPE'  
    if title == 'Possui Chave':
        return 'HAS-KEY'

In [5]:
def get_element_text(navigator, path):
    value = navigator.find_element(By.XPATH, os.getenv(path)).text
    return value

In [6]:
def get_vehicle_data():
    title = []
    value = []
    for i in range(1,10):
        try:     
            result = navigator.find_element(By.XPATH,f"/html/body/div[2]/div/div[1]/main/div[2]/div[2]/div[3]/div[2]/div/div[2]/div/div[2]/div[1]/div[{i}]/div/div").text.split('\n')
            title.append(result[0])                
            value.append(result[1])
        except:
            pass

    data = {translate_title(title[i]): value[i] for i in range(len(title))}
    return data

##### Get Title

In [7]:
auction_title = get_element_text(navigator, 'AUCTION-TITLE').replace('/','_')
auction_title

'LEILÃO DE VEÍCULOS NORDESTE - 08_03_23'

In [8]:
time.sleep(5)
total_results = int(get_element_text(navigator, 'TOTAL-RESULTS').split(' ')[0])
total_results

109

#### Click on car

In [9]:
time.sleep(5)
click(os.getenv('FIRST-CAR'))
time.sleep(5)

In [10]:
def write_csv(name, cars):
    with open(f'{name}.csv', 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames = cars[0].keys())
        writer.writeheader()
        writer.writerows(cars)

In [11]:
def get_information(navigator):
    car_info = get_vehicle_data()
    full_car_info = {'BATCH': get_element_text(navigator, 'BATCH'),
        'VEHICLE': get_element_text(navigator, 'VEHICLE'),
        'BANK':get_element_text(navigator, 'BANK'),
        'KM':'',
        'YEAR': '',
        'FUEL': '',
        'COLOR': '',
        'FIPE': '',
        'YARD': '',
        'RESUME-TYPE': '',
        'TYPE':'',
        'HAS-KEY': ''
        }
    full_car_info.update(car_info)
    return full_car_info

In [12]:
cars = []

for _ in range(1, total_results+1):
    try:
        data = get_information(navigator)
        cars.append(data)
        time.sleep(5)
        click(os.getenv('NEXT_VEHICLE')) 
    except:
        print('erro')
        click(os.getenv('NEXT_VEHICLE')) 

In [13]:
cars

[{'BATCH': '1',
  'VEHICLE': 'renault/kwid zen 1.0 mecanico',
  'BANK': 'BANCO SANTANDER S/A',
  'KM': '23.368',
  'YEAR': '20/21',
  'FUEL': 'Flex',
  'COLOR': 'BRANCA',
  'FIPE': 'R$ 49.991,00',
  'YARD': 'PATIO 2 - FORTALEZA/CE',
  'RESUME-TYPE': 'Recuperado de Financiamento',
  'TYPE': 'Veículos',
  'HAS-KEY': 'Sim'},
 {'BATCH': '2',
  'VEHICLE': 'chevrolet/prisma joye 1.0 mecanico (4p)',
  'BANK': 'BANCO SANTANDER S/A',
  'KM': '58.003',
  'YEAR': '19/19',
  'FUEL': 'Flex',
  'COLOR': 'BRANCA',
  'FIPE': 'R$ 53.894,00',
  'YARD': 'PATIO 2 - FORTALEZA/CE',
  'RESUME-TYPE': 'Recuperado de Financiamento',
  'TYPE': 'Veículos',
  'HAS-KEY': 'Sim'},
 {'BATCH': '3',
  'VEHICLE': 'fiat/strada hard working cabine simples cc e',
  'BANK': 'BANCO SANTANDER S/A',
  'KM': '102.032',
  'YEAR': '19/19',
  'FUEL': 'Flex',
  'COLOR': 'PRATA',
  'FIPE': 'R$ 61.111,00',
  'YARD': 'PATIO 2 - FORTALEZA/CE',
  'RESUME-TYPE': 'Recuperado de Financiamento',
  'TYPE': 'Veículos',
  'HAS-KEY': 'Sim'},
 {'

In [14]:
write_csv(name = auction_title, cars = cars)

In [5]:
vehicles = pd.read_csv('LEILÃO DE VEÍCULOS NORDESTE - 08_03_23.csv')

In [10]:
vehicles.head(2)

,BATCH,VEHICLE,BANK,KM,YEAR,FUEL,COLOR,FIPE,YARD,RESUME-TYPE,TYPE,HAS-KEY
0,1,renault/kwid zen 1.0 mecanico,BANCO SANTANDER S/A,23.368,20/21,Flex,BRANCA,"R$ 49.991,00",PATIO 2 - FORTALEZA/CE,Recuperado de Financiamento,Veículos,Sim
1,2,chevrolet/prisma joye 1.0 mecanico (4p),BANCO SANTANDER S/A,58.003,19/19,Flex,BRANCA,"R$ 53.894,00",PATIO 2 - FORTALEZA/CE,Recuperado de Financiamento,Veículos,Sim


In [11]:
vehicles.sort_values(by='KM').head(10)

,BATCH,VEHICLE,BANK,KM,YEAR,FUEL,COLOR,FIPE,YARD,RESUME-TYPE,TYPE,HAS-KEY
66,139,honda/pcx 150 sport abs,BANCO SANTANDER S/A,8.414,20/21,Gasolina,PRATA,"R$ 18.164,00",PATIO 2 - FORTALEZA/CE,Recuperado de Financiamento,Motos,Sim
57,120,honda/cg 160 fan,BANCO SANTANDER S/A,11.884,22/22,Flex,VERMELHA,"R$ 15.772,00",PATIO 2 - FORTALEZA/CE,Recuperado de Financiamento,Motos,Sim
59,125,haojian/avelloz az1,BANCO SANTANDER S/A,11.922,21/22,Gasolina,BRANCA,"R$ 7.499,00",PATIO 2 - FORTALEZA/CE,Recuperado de Financiamento,Motos,Sim
65,137,kawasak/z400,BANCO SANTANDER S/A,16.192,21/21,Gasolina,PRETA,"R$ 28.924,00",PATIO 2 - FORTALEZA/CE,Recuperado de Financiamento,Motos,Sim
0,1,renault/kwid zen 1.0 mecanico,BANCO SANTANDER S/A,23.368,20/21,Flex,BRANCA,"R$ 49.991,00",PATIO 2 - FORTALEZA/CE,Recuperado de Financiamento,Veículos,Sim
28,34,volkswagen/gol mc4 (4p),BANCO VOLKSWAGEN S/A,30.498,19/20,Flex,VERMELHA,"R$ 51.823,00",PATIO FORTALEZA/CE,Recuperado de Financiamento,Veículos,Sim
22,28,volkswagen/polo comfortline ad,BANCO VOLKSWAGEN S/A,50.641,19/20,Flex,AZUL,"R$ 84.521,00",PATIO FORTALEZA/CE,Recuperado de Financiamento,Veículos,Sim
96,213,ford/fiesta flex (4p),BANCO SANTANDER S/A,51.049,13/14,Flex,BRANCA,"R$ 30.046,00",PATIO 2 - FORTALEZA/CE,Recuperado de Financiamento,Veículos,Sim
5,6,fiat/argo drive 1.0 (4p),BANCO SANTANDER S/A,53.201,18/18,Flex,VERMELHA,"R$ 54.644,00",PATIO 2 - FORTALEZA/CE,Recuperado de Financiamento,Veículos,Sim
1,2,chevrolet/prisma joye 1.0 mecanico (4p),BANCO SANTANDER S/A,58.003,19/19,Flex,BRANCA,"R$ 53.894,00",PATIO 2 - FORTALEZA/CE,Recuperado de Financiamento,Veículos,Sim
